## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [3]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [18]:
load_dotenv(override=True)
openai = OpenAI(api_key=os.getenv("deepseek_api_key"), base_url="https://api.deepseek.com")
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

answer = openai.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ]
)

print(answer.choices[0].message.content)

The capital of France is **Paris**. It is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.  

Would you like information on anything else related to Paris or France? 😊


In [5]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
jbtejasvi@gmail.com
www.linkedin.com/in/tejasvi-jb
(LinkedIn)
Top Skills
GitHub
Git
Bootstrap (Framework)
Certifications
SQL Associate
Learn Responsive Design Course
Cutshort Certified Javascript -
Advanced
Tejasvi Jb
Front End | Hids Technologies
Bengaluru, Karnataka, India
Summary
Hello! I am a passionate frontend developer with a love for clean
code and elegant design. When I'm not crafting user-friendly
interfaces, you can find me exploring the great outdoors on two
wheels ‍♂️️. Let's connect and create something awesome together!
Experience
HiDs Technologies
Frontend Developer
November 2023 - Present (1 year 8 months)
KYC Hub
SDE - Front End 
December 2022 - September 2023 (10 months)
London, England, United Kingdom
Education
Impact College Of Engineering And Applied Sciences
Bachelor of Engineering - BE, Computer Science · (2021)
  Page 1 of 1


In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Tejasvi jb"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Tejasvi jb. You are answering questions on Tejasvi jb's website, particularly questions related to Tejasvi jb's career, background, skills and experience. Your responsibility is to represent Tejasvi jb for interactions on the website as faithfully as possible. You are given a summary of Tejasvi jb's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n👋 Hello! I’m a passionate Frontend Developer who loves building fast, responsive, and accessible web applications. I specialize in HTML, CSS, JavaScript, and modern frameworks like React, Next.js, and TypeScript. I have a strong eye for UI/UX design, and I focus on crafting clean, maintainable code with a seamless user experience.\n\nI'm experienced with state management tools like Redux, data fetching libraries like React Query, and sty

In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

In [20]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [21]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [22]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [24]:
import os
openai = OpenAI(api_key=os.getenv("deepseek_api_key"), base_url="https://api.deepseek.com")


In [41]:
import json
import re

def extract_json_from_markdown(text):
    # This regex finds a code block with optional 'json' after the backticks
    match = re.search(r"```(?:json)?\s*([\s\S]*?)```", text)
    if match:
        return match.group(1)
    return text  # fallback: return as-is if no code block found

def evaluate(reply, message, history) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    # Ask the model to reply in JSON
    messages[-1]["content"] += (
        "\n\nPlease reply in the following JSON format:\n"
        '{"is_acceptable": true/false, "feedback": "your feedback here"}'
    )
    response = openai.chat.completions.create(model="deepseek-chat", messages=messages)
    content = response.choices[0].message.content
    # Try to parse the JSON from the response
    json_str = extract_json_from_markdown(content)

    try:
        data = json.loads(json_str)
        return Evaluation(**data)
    except Exception as e:
        print("Failed to parse model response:", e)
        print("Model response was:", content)
        # Fallback: return a default Evaluation or raise
        return Evaluation(is_acceptable=False, feedback="Could not parse model response.")

In [32]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="deepseek-chat", messages=messages)
reply = response.choices[0].message.content

print(reply)

Hi there!  

Thanks for your interest in my work. Currently, I don't hold any patents—my focus has been on building and refining frontend applications with modern web technologies. However, I'm always exploring innovative solutions in UI/UX, performance optimization, and AI-driven web experiences. If I ever venture into patent-worthy work, I’ll be sure to share it!  

If you're curious about any specific projects or technologies I've worked on, feel free to ask—I'd be happy to discuss them. 🚀  

Best,  
Tejasvi


In [29]:
reply

'Hi there!  \n\nGreat question—I don’t currently hold any patents, but I’m always exploring innovative solutions in frontend development, AI integration, and web performance optimization. If an idea comes along that’s truly groundbreaking, I’d love to pursue that path!  \n\nFor now, my focus is on building high-quality, scalable applications with modern tech like React, Next.js, and TypeScript. If you’re curious about any of my projects or contributions, feel free to ask!  \n\nWould love to hear what sparked your interest—are you working on something patent-worthy yourself? 🚀'

In [42]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is professional, engaging, and accurately represents Tejasvi jb's background. It clearly states that there are no patents held, which is honest and transparent. The agent also redirects the conversation positively by inviting questions about specific projects or technologies, maintaining a helpful and approachable tone. The use of emojis and the closing signature align well with the friendly yet professional persona described in the summary.")

In [43]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="deepseek-chat", messages=messages)
    return response.choices[0].message.content

In [44]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="deepseek-chat", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [45]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable as it switches to Pig Latin, which is unprofessional and confusing for a potential client or employer. The Agent should maintain a professional tone and provide a clear, straightforward answer. The playful tone and language switch detract from the credibility and professionalism expected in this context.
